In [2]:
import collections
from numpy.lib.function_base import vectorize
import pandas as pd 
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy import sparse
from scipy.spatial import distance
import sklearn.metrics
import sklearn.neighbors
import sklearn.linear_model
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from better_profanity import profanity
import time
import os
import itertools

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [3]:
# loading user  data 

# loading training data .jsonl
TRAINING_DATA = './resources/data/train.jsonl'
VAL_DATA = './resources/data/val.jsonl'

df_train, df_val = pd.read_json(TRAINING_DATA, lines=True), pd.read_json(VAL_DATA, lines=True)
USER_DATA = './resources/data/users.json'
df_user = pd.read_json(USER_DATA, orient="index")

# Getting User Features
TODO: 
1. Get the religious ideology, political ideology for both the debaters and judges 
2. Get the matching - 4 features
    1. religious match with pro / con 
    2. political match with pro / con 
3. Get cosaine similarity of big issue opinions - 2 features 
    1. get the vector to represent that 
        1. {'N/O', 'Und', 'Pro', 'N/S', 'Con'}
        2. have one vector each of the issue
    1. similarity of each voter with pro / con, the find the difference of the average 

In [261]:
# The users.json file contains a json object with the following fields:

# ‘big_issues_dict’ — The user’s stance on what are considered the big issues on this debating forum (such as abortion, death penalty, gun rights, etc.).
#    59% has no missing value 
# ‘joined’ — The date the user joined the platform.
# ‘opinion_arguments’ — The user’s response to opinion questions that are posted by others on the platform.
# ‘opinion_questions’, — The opinion questions posted by the user.
# ‘party' — The political party that the user associates with.
#   choose the one that exist 
#    25%
# 'political_ideology' — The user’s political ideology.
#   choose the one that exist 
#    27%
# 'poll_topics’ — Polls posted by the user.
# ’poll_votes' — The user’s response to polls posted by other users on the platform.
#    27%
# ‘relationship' — The user’s relationship status.
# ‘religious_ideology' — The user’s religious ideology.
#    33%


# Construct 5 x # issue dimension vector for all issues for a given voter

# input to the transformer 
#         return {"Pro": 
#                         {"df": df,
#                         "unigram_vectorizer": self.unigram_vectorizer,
#                         "trigram_vectorizer": self.trigram_vectorizer,
#                         "document": document_side[:, 0],
#                         "side": "Pro"},
#                "Con": 
#                         {"df": df,
#                         "unigram_vectorizer": self.unigram_vectorizer,
#                         "trigram_vectorizer": self.trigram_vectorizer,
#                         "document": document_side[:, 1],
#                         "side": "Con"}
#                }
        
print(transformer_get_political_align.df_user.columns)
collections.Counter(transformer_get_political_align.df_user.birthday)

Index(['big_issues_dict', 'birthday', 'education', 'ethnicity', 'gender',
       'friends', 'income', 'joined', 'opinion_arguments', 'opinion_questions',
       'party', 'political_ideology', 'poll_topics', 'poll_votes',
       'relationship', 'religious_ideology'],
      dtype='object')


Counter({'- Private -': 44682,
         'January 2, 1992': 1,
         'January 9, 2000': 1,
         'August 25, 1997': 1,
         'December 14, 2003': 1,
         'August 14, 2000': 1,
         'March 4, 1998': 1,
         'March 25, 1996': 1,
         'February 5, 1995': 2,
         'July 22, 1996': 1,
         'March 15, 1999': 1,
         'September 1, 1997': 1,
         'July 31, 1986': 1,
         'April 14, 1960': 1,
         'October 28, 1999': 1,
         'June 9, 1998': 1,
         'October 28, 1996': 1,
         'November 19, 1995': 1,
         'October 15, 1991': 1,
         'December 27, 1983': 1,
         'October 31, 1999': 1,
         'January 19, 1999': 1,
         'July 19, 1990': 1,
         'October 1, 2000': 1,
         'January 17, 1995': 1,
         'August 30, 1992': 1,
         'October 16, 1998': 1,
         'June 7, 1999': 2,
         'September 13, 1995': 1,
         'December 27, 1992': 1,
         'March 13, 1965': 1,
         'June 6, 1994': 1,
        

# Getting Linguistic Features

In [5]:
# ========================================= Get user features ==============================================
class Transformer_get_any_user_align(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.category = list(set(df_user.political_ideology))
        self.religous_type = CategoricalDtype(self.category, ordered=True)
        self.column = column
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        df = X["Pro"]['df']
        voters_political = df.loc[:, 'voters'].apply(self.get_political_voters)
        
        pro_political = df.loc[:, 'pro_debater'].apply(self.get_political)
        pro_political_align = pd.DataFrame({'user_{}'.format(self.column): pro_political,
                                           'voters_{}'.format(self.column): voters_political}).apply(self.get_match_political, 
                                                                                       axis=1)
        
        con_political = df.loc[:, 'con_debater'].apply(self.get_political)
        con_political_align = pd.DataFrame({'user_{}'.format(self.column): con_political,
                                           'voters_{}'.format(self.column): voters_political}).apply(self.get_match_political,
                                                                                       axis=1)
        
        return np.hstack([np.vstack(pro_political_align.values), np.vstack(con_political_align.values)])
        
        
    def get_political(self, user): 
        user_political = df_user.loc[user, self.column]
        return user_political

    def get_political_voters(self, voters): 
        political_vectors = []
        voters = np.array(voters)
        eligible_voters = voters[list(map(lambda voter: voter in df_user.index, voters))]
        if len(eligible_voters) > 0:
            data = np.array(list(map(self.get_political, eligible_voters)))
        else:
            data = np.nan
        return data

    def get_match_political(self, row): 
        user_political, voters_political = row["user_{}".format(self.column)], row["voters_{}".format(self.column)]

        if voters_political is np.nan:
            return [0, 0, 0]

        if user_political == 'Not Saying':
            return np.array([0, 1, 0])

        feature = np.array([0, 0, 0])
        for v_r in voters_political:
            if v_r == 'Not Saying' :
                feature += np.array([0, 1, 0])
            elif v_r == user_political:
                feature += np.array([0, 0, 1])
            else:
                feature += np.array([1, 0, 0])

        return feature / np.sum(feature)

class Transformer_get_political_align(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.category = list(set(df_user.political_ideology))
        self.religous_type = CategoricalDtype(self.category, ordered=True)
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        df = X["Pro"]['df']
        voters_political = df.loc[:, 'voters'].apply(self.get_political_voters)
        
        pro_political = df.loc[:, 'pro_debater'].apply(self.get_political)
        pro_political_align = pd.DataFrame({'user_political': pro_political,
                                           'voters_political': voters_political}).apply(self.get_match_political, 
                                                                                       axis=1)
        
        con_political = df.loc[:, 'con_debater'].apply(self.get_political)
        con_political_align = pd.DataFrame({'user_political': con_political,
                                           'voters_political': voters_political}).apply(self.get_match_political,
                                                                                       axis=1)
        
        return np.hstack([np.vstack(pro_political_align.values), np.vstack(con_political_align.values)])
        
        
    def get_political(self, user): 
        user_political = df_user.loc[user, "political_ideology"]
        return user_political

    def get_political_voters(self, voters): 
        political_vectors = []
        voters = np.array(voters)
        eligible_voters = voters[list(map(lambda voter: voter in df_user.index, voters))]
        if len(eligible_voters) > 0:
            data = np.array(list(map(self.get_political, eligible_voters)))
        else:
            data = np.nan
        return data

    def get_match_political(self, row): 
        user_political, voters_political = row["user_political"], row["voters_political"]

        if voters_political is np.nan:
            return [0, 0, 0]

        if user_political == 'Not Saying':
            return np.array([0, 1, 0])

        feature = np.array([0, 0, 0])
        for v_r in voters_political:
            if v_r == 'Not Saying' :
                feature += np.array([0, 1, 0])
            elif v_r == user_political:
                feature += np.array([0, 0, 1])
            else:
                feature += np.array([1, 0, 0])

        return feature / np.sum(feature)

class Transformer_get_religious_align(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.category = list(set(df_user.religious_ideology))
        self.religous_type = CategoricalDtype(self.category, ordered=True)
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        df = X["Pro"]['df']
        voters_religious = df.loc[:, 'voters'].apply(self.get_religious_voters)
        
        pro_religious = df.loc[:, 'pro_debater'].apply(self.get_religious)
        pro_religious_align = pd.DataFrame({'user_religious': pro_religious,
                                           'voters_religious': voters_religious}).apply(self.get_match_religious, 
                                                                                       axis=1)
        
        con_religious = df.loc[:, 'con_debater'].apply(self.get_religious)
        con_religious_align = pd.DataFrame({'user_religious': con_religious,
                                           'voters_religious': voters_religious}).apply(self.get_match_religious,
                                                                                       axis=1)
        
        return np.hstack([np.vstack(pro_religious_align.values), np.vstack(con_religious_align.values)])
        
        
    def get_religious(self, user): 
        user_religious = df_user.loc[user, "religious_ideology"]
        return user_religious

    def get_religious_voters(self, voters): 
        religious_vectors = []
        voters = np.array(voters)
        eligible_voters = voters[list(map(lambda voter: voter in df_user.index, voters))]
        if len(eligible_voters) > 0:
            data = np.array(list(map(self.get_religious, eligible_voters)))
        else:
            data = np.nan
        return data

    def get_match_religious(self, row): 
        user_religious, voters_religious = row["user_religious"], row["voters_religious"]

        if voters_religious is np.nan:
            return [0, 0, 0]

        if user_religious == 'Not Saying':
            return np.array([0, 1, 0])

        feature = np.array([0, 0, 0])
        for v_r in voters_religious:
            if v_r == 'Not Saying' :
                feature += np.array([0, 1, 0])
            elif v_r == user_religious:
                feature += np.array([0, 0, 1])
            else:
                feature += np.array([1, 0, 0])

        return feature / np.sum(feature)

        



class Transformer_get_cosine_similarity(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.cache = {}
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        key = str(len(X['Pro']['document']))
        if key in self.cache:
            return self.cache[key]
        
        df = X["Pro"]['df']
        
        big_issues_vector_con = df.loc[:, "con_debater"].apply(self.get_big_issues_vector)
        big_issues_vector_pro = df.loc[:, "pro_debater"].apply(self.get_big_issues_vector)
        big_issues_vector_voters = df.loc[:, "voters"].apply(self.get_big_issues_vector_voters)
        big_issues_df = pd.DataFrame.from_dict({
            "pro": big_issues_vector_pro, 
            "con": big_issues_vector_con, 
            "voters": big_issues_vector_voters
        })
        similarity_df = big_issues_df.apply(self.get_cosine_similarity_voters, axis=1)
        
        similarity_feature = np.hstack([similarity_df[['avg_similarity_pro']].values, 
                                        similarity_df[['avg_similarity_con']].values])
        self.cache[key] = similarity_feature
        return similarity_feature
        

    def get_big_issues_vector(self, user): 
        if user not in df_user.index:
            return np.nan
        user_big_issues_dict = df_user.loc[user, "big_issues_dict"]
        opinion_type = CategoricalDtype(['N/O', 'Und', 'Pro', 'N/S', 'Con'], ordered=True)
        user_big_issues_df = pd.DataFrame(data={"opinion": user_big_issues_dict.values()}, 
                                          index=user_big_issues_dict.keys(),
                                          dtype=opinion_type)


        vector = np.concatenate(pd.get_dummies(user_big_issues_df).values, axis=0)
        return vector

    def get_big_issues_vector_voters(self, voters): 
        big_issues_vectors = []
        voters = np.array(voters)
        eligible_voters = voters[list(map(lambda voter: voter in df_user.index, voters))]
        if len(eligible_voters) > 0:
            data = np.vstack(list(map(self.get_big_issues_vector, eligible_voters)))
        else:
            data = np.nan
        return data


    def get_cosine_similarity_voters(self, row):
        con = row['con']
        pro = row['pro']
        voters = row['voters']
        
        if np.isnan(np.sum(voters)):
            return pd.Series([np.nan, np.nan, 0.5, 0.5],
                            index=["similarity_pro", "similarity_con", "avg_similarity_pro", "avg_similarity_con"])

        if con is np.nan:
            con_similarity = np.nan
            con_avg_similarity = 0.5
        else:
            con_similarity = np.array(list(map(lambda voter: distance.cosine(voter, con), voters)))
            con_avg_similarity = np.average(con_similarity)
        if pro is np.nan:
            pro_similarity = np.nan
            pro_avg_similarity = 0.5
        else:
            pro_similarity = np.array(list(map(lambda voter: distance.cosine(voter, pro), voters)))
            pro_avg_similarity = np.average(pro_similarity)
            
        return pd.Series([pro_similarity, con_similarity, pro_avg_similarity, con_avg_similarity],
                        index=["similarity_pro", "similarity_con", "avg_similarity_pro", "avg_similarity_con"])
    
# ========================================= Get linguistic features ==============================================
class Transformer_identity(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        return X

class Transformer_get_length(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None): 
        '''X should have two columns. One is the document and ther other is the corresponding unigram vector'''
        # Count the number if unigrams in a feature
        unigram = X["unigram"]
        
        length = unigram.sum(axis=1)
        return length
class Transformer_get_reference_to_opponent(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None): 
        # Count the number of times the opponent's username is mentioned 
        df = X["df"]
        
        count = df.apply(self.count_opponent, axis=1).values
        count = np.reshape(count, newshape=[-1, 1])
        
        return count 
    
    def count_opponent(self, row):
        document_lower = row["document"].lower() 
        count = document_lower.count(row["opponent"]) + document_lower.count("opponent")
        
        return count
            
class Transformer_get_swear_words(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.matrix = None
    def fit(self, X, y=None):
        unigram_vectorizer = X['unigram_vectorizer']
        vector = list(map(lambda x: int(profanity.contains_profanity(x)), 
                                unigram_vectorizer.get_feature_names()))

        self.matrix = np.reshape(vector, newshape=[-1, 1])
        return self 

    def transform(self, X, y=None):
    #     perhaps get rid some of the swear words because they look like they are necessary words 
    #     for discussion such as arian, sodom 
        unigram = X['unigram']
        unigram_vectorizer = X['unigram_vectorizer']
        

        swear_pro = unigram @ self.matrix
        return swear_pro

class Transformer_get_personal_pronouns(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.personal_pronouns = pd.Series(
            ["I", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them"])
        self.matrix = None
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
    #     ================== faster but less accurate ========================

        document = X['df'].loc[:, 'document']

        all_counts = []

        for name in self.personal_pronouns:
            count = np.array(list(map(lambda x: x.count(" {} ".format(name)), document)))
            count = np.reshape(count, newshape=[-1, 1])
            all_counts.append(count)


        personal_pronouns_feature = np.hstack(all_counts)
        return personal_pronouns_feature

    #     ================== more accurate but slower ========================
        # personal_pronouns_vector = unigram_vectorizer.transform(personal_pronouns)
        # matrix_person_pronouns = personal_pronouns_vector.T 

        # document_pro = document_side[:, 0]
        # unigram_pro = vectorizer.transform(document_pro)
        # personal_pronouns_feature_pro = unigram_pro @ matrix_person_pronouns
        # I_count_pro = np.array(list(map(lambda x: x.count(" I "), document_pro)))
        # I_count_pro = np.reshape(I_count_pro, newshape=[-1, 1])
        # personal_pronouns_feature_pro = sparse.hstack([personal_pronouns_feature_pro, I_count_pro])

        # document_con = document_side[:, 1]
        # unigram_con = vectorizer.transform(document_con)
        # personal_pronouns_feature_con = unigram_con @ matrix_person_pronouns
        # I_count_con = np.array(list(map(lambda x: x.count(" I "), document_con)))
        # I_count_con = np.reshape(I_count_con, newshape=[-1, 1])
        # personal_pronouns_feature_con = sparse.hstack([personal_pronouns_feature_con, I_count_con])
        
class Transformer_get_questions(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        document = X['df']['document']

        question_count = np.array(list(map(lambda x: x.count("?"), document)))
        question_count = np.reshape(question_count, newshape=[-1, 1])
        return question_count


class Transformer_get_websites(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        document = X['df']['document']
        website_count = np.array(list(map(lambda x: x.count("http"), document)))
        website_count = np.reshape(website_count, newshape=[-1, 1])
        return website_count

class Transformer_get_exclamation(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):

        document = X['df']['document']
        exclamation_count = np.array(list(map(lambda x: x.count("!"), document)))
        exclamation_count = np.reshape(exclamation_count, newshape=[-1, 1])
        return exclamation_count

class Transformer_get_number(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.matrix_number = None
        pass
    def fit(self, X, y=None):
        unigram_vectorizer = X['unigram_vectorizer']
        vector_number = list(map(lambda x: int(x[0].isnumeric()), unigram_vectorizer.get_feature_names()))
        self.matrix_number = np.reshape(vector_number, newshape=[-1, 1])
        return self
    def transform(self, X, y=None):
        document = X['df']['document']
        unigram = X['unigram']
        number = unigram @ self.matrix_number

        return number

class Transformer_get_modal_verb(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.matrix_modal_verb = None
        pass
    def fit(self, X, y=None):
        unigram_vectorizer = X['unigram_vectorizer']
        modal_verbs = set(["can", "could", "may", "might", "shall", "should", "will", "would", "must"])
        vector_modal_verb = list(map(lambda x: int(x in modal_verbs), unigram_vectorizer.get_feature_names()))
        self.matrix_modal_verb = np.reshape(vector_modal_verb, newshape=[-1, 1])
        return self
    
    def transform(self, X, y=None):

        document = X['df']['document']
        unigram = X['unigram']
        modal_verb = unigram @ self.matrix_modal_verb

        return modal_verb

# ========================================= Get Lexicon features ==============================================
class Transformer_get_vad_feature(BaseEstimator, TransformerMixin):
    def __init__(self):
        NRC_LEXICON_VAD = "./resources/lexica/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt"
        
        self.df_vad = pd.read_csv(NRC_LEXICON_VAD, sep="	", header=None)
        self.df_vad.columns = ["word", "valence", "arousal", "dominance"]
        self.df_vad = self.df_vad.dropna()
        self.df_vad = self.df_vad.set_index("word")
        self.df_vad["valence"] = self.df_vad["valence"].astype('category')
        self.df_vad["arousal"] = self.df_vad["arousal"].astype('category')
        self.df_vad["dominance"] = self.df_vad["dominance"].astype('category')
        
    def fit(self, X, y=None):
        unigram_vectorizer = X['unigram_vectorizer']
        
        word_vad = self.df_vad.index
        word_vector_vad = unigram_vectorizer.transform(word_vad)
        self.matrix_vad = word_vector_vad.T @ self.df_vad 
        return self 
    
    def transform(self, X, y=None):
        gram_pro = X['unigram']
        feature_pro = gram_pro @ self.matrix_vad / np.sum(gram_pro)
        return feature_pro
        
class Transformer_get_connotation_feature(BaseEstimator, TransformerMixin):
    def __init__(self):
        CONNOTATION = "./resources/lexica/connotation_lexicon_a.0.1.csv"

        self.df_connotation = pd.read_csv(CONNOTATION, sep=",|_", header=None)
        self.df_connotation.columns = ["word", "pos", "connotation"] # word, part of speech, connotation
        self.df_connotation = self.df_connotation.dropna() # There are five words in the connotation that are nan 
        self.df_connotation = self.df_connotation.set_index("word")
        self.df_connotation["pos"] = self.df_connotation["pos"].astype('category')
        self.df_connotation = self.df_connotation.drop(columns=["pos"]) # drop the part of speech classification because we can't use it now 
        self.df_connotation["connotation"] = self.df_connotation["connotation"].astype('category')
        self.df_connotation = pd.get_dummies(self.df_connotation)
    
    def fit(self, X, y=None):
        unigram_vectorizer = X['unigram_vectorizer']
        
        word_connotation = self.df_connotation.index
        word_vector_connotation = unigram_vectorizer.transform(word_connotation)
        self.matrix_connotation = word_vector_connotation.T @ self.df_connotation
        return self 
    
    def transform(self, X, y=None):
        gram_pro = X['unigram']
        feature_pro = gram_pro @ self.matrix_connotation / np.sum(gram_pro)
        return feature_pro
    
# ========================================= Get ngrams features ==============================================
class Transformer_get_trigrams(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.existing_trigram = {}
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        key = X['side'] + str(len(X['df']['document']))
        if key in self.existing_trigram:
            return self.existing_trigram[key]
        
        document = X['df']['document'] 
        trigram_vectorizer = X['trigram_vectorizer']
        
        trigram = trigram_vectorizer.transform(document)
        self.existing_trigram[key] = trigram
        
        return trigram

class Transformer_get_unigrams(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        document = X['document'] 
        df = X['df'] 
        side = X['side']
        opponent = 'pro_debater' if side == 'Con' else 'con_debater'
        unigram_vectorizer = X['unigram_vectorizer']
        trigram_vectorizer = X['trigram_vectorizer']
        
        df_side = pd.DataFrame.from_dict(
            {
                "document": document,
                "opponent": df.loc[:, opponent]
            }
        )

        feature = {"df": df_side, 
                    "unigram": unigram_vectorizer.transform(document),
                    "unigram_vectorizer": unigram_vectorizer,
                    "trigram_vectorizer": trigram_vectorizer,
                    "side": side
                    }
        
        return feature

# ========================================= Preprocessing ==============================================
class Transformer_separate_document(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.unigram_vectorizer = None
        self.trigram_vectorizer = None
        self.document_side = None
        self.document = None
        
    def fit(self, df, y=None):
        document_side = get_text_by_side(df)
        document = [side[0] + side[1] for side in document_side]
        
        if not self.unigram_vectorizer:
            self.unigram_vectorizer = CountVectorizer()
            self.unigram_vectorizer.fit(document)
            
        if not self.trigram_vectorizer:
            self.trigram_vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.8, 
                                                 min_df=0.2, stop_words='english', ngram_range=(1,3))
            self.trigram_vectorizer.fit(document)
            
        return self 
    
    def transform(self, df, y=None):
        document_side = get_text_by_side(df)
        document = [side[0] + side[1] for side in document_side]
        
        return {"Pro": 
                        {"df": df,
                        "unigram_vectorizer": self.unigram_vectorizer,
                        "trigram_vectorizer": self.trigram_vectorizer,
                        "document": document_side[:, 0],
                        "side": "Pro"},
               "Con": 
                        {"df": df,
                        "unigram_vectorizer": self.unigram_vectorizer,
                        "trigram_vectorizer": self.trigram_vectorizer,
                        "document": document_side[:, 1],
                        "side": "Con"}
               }
        

transformer_get_length = Transformer_get_length()
transformer_identity = Transformer_identity()
transformer_get_reference_to_opponent = Transformer_get_reference_to_opponent()
transformer_get_swear_words = Transformer_get_swear_words()
transformer_get_personal_pronouns = Transformer_get_personal_pronouns()

transformer_get_questions = Transformer_get_questions()
transformer_get_websites = Transformer_get_websites()
transformer_get_exclamation = Transformer_get_exclamation()
transformer_get_number = Transformer_get_number()
transformer_get_modal_verb = Transformer_get_modal_verb()

transformer_separate_document = Transformer_separate_document()
transformer_get_unigrams = Transformer_get_unigrams()
transformer_get_trigrams = Transformer_get_trigrams()
transformer_get_connotation_feature = Transformer_get_connotation_feature()
transformer_get_vad_feature = Transformer_get_vad_feature()

transformer_get_cosine_similarity = Transformer_get_cosine_similarity()
transformer_get_religious_align = Transformer_get_religious_align()
transformer_get_political_align = Transformer_get_political_align()
transformer_get_education_user_align = Transformer_get_any_user_align('education')
transformer_get_party_user_align = Transformer_get_any_user_align('party')
transformer_get_gender_user_align = Transformer_get_any_user_align('gender')
transformer_get_ethnicity_user_align = Transformer_get_any_user_align('ethnicity')
transformer_get_relationship_user_align = Transformer_get_any_user_align('relationship')

transformer_get_pro = FunctionTransformer(lambda both_side_X: both_side_X['Pro'])
transformer_get_con = FunctionTransformer(lambda both_side_X: both_side_X['Con'])

linguistic_trans = FeatureUnion(
    [
        ('length', transformer_get_length),
        ('reference_to_opponent', transformer_get_reference_to_opponent),
        ('swear_words', transformer_get_swear_words),
        ('personal_pronouns', transformer_get_personal_pronouns), 
        ("questions",  transformer_get_questions),
        ("websites",  transformer_get_websites),
        ("exclamation",  transformer_get_exclamation),
        ("number",  transformer_get_number),
        ("modal_verb",   transformer_get_modal_verb),
        
        ('connotation', transformer_get_connotation_feature),
        ('vad', transformer_get_vad_feature),
        
        ('trigram', transformer_get_trigrams)
    ]
)

side_trans = Pipeline(
    [
        ('ngram', transformer_get_unigrams), 
        ('linguistic', linguistic_trans)
    ]
)

pro_trans = Pipeline(
    [
        ('get_pro', transformer_get_pro),
        ('side_trans', side_trans) 
    ]
)

con_trans = Pipeline(
    [
        ('get_con', transformer_get_con),
        ('side_trans', side_trans) 
    ]
)

both_trans = FeatureUnion(
    [
#         ('pro_feature', pro_trans),
#         ('con_feature', con_trans),
#         ('religious_align', transformer_get_religious_align)
#         ('political_align', transformer_get_political_align)
        ('education_align', transformer_get_education_user_align)
#         ('cosine_similarity', transformer_get_cosine_similarity)
    ]
)
                
big_trans = Pipeline(
    [
        ('separate_document', transformer_separate_document), 
        ('get_both_features', both_trans),
        ('logistic_regression', sklearn.linear_model.LogisticRegression())
    ]
)


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/ipykernel_launcher.py:470: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [255]:
clf = big_trans.fit(df_train, get_winner(df_train))
print(classification_report(get_winner(df_train), 
                           clf.predict(df_train)))
report = classification_report(get_winner(df_val), 
                           clf.predict(df_val), output_dict=True)

              precision    recall  f1-score   support

           0       0.61      0.87      0.72       916
           1       0.59      0.26      0.36       676

    accuracy                           0.61      1592
   macro avg       0.60      0.56      0.54      1592
weighted avg       0.60      0.61      0.57      1592



In [6]:
print()

In [242]:
report

{'0': {'precision': 0.557632398753894,
  'recall': 0.8483412322274881,
  'f1-score': 0.6729323308270676,
  'support': 211},
 '1': {'precision': 0.5897435897435898,
  'recall': 0.24468085106382978,
  'f1-score': 0.3458646616541353,
  'support': 188},
 'accuracy': 0.5639097744360902,
 'macro avg': {'precision': 0.5736879942487418,
  'recall': 0.546511041645659,
  'f1-score': 0.5093984962406015,
  'support': 399},
 'weighted avg': {'precision': 0.5727624837314951,
  'recall': 0.5639097744360902,
  'f1-score': 0.5188252586353101,
  'support': 399}}

# Feature selection

In [27]:
for i in itertools.combinations(linguistic_transfomer_dict, 2):
    print(i)

('length', 'reference_to_opponent')
('length', 'swear_words')
('length', 'personal_pronouns')
('length', 'questions')
('length', 'websites')
('length', 'exclamation')
('length', 'number')
('length', 'modal_verb')
('reference_to_opponent', 'swear_words')
('reference_to_opponent', 'personal_pronouns')
('reference_to_opponent', 'questions')
('reference_to_opponent', 'websites')
('reference_to_opponent', 'exclamation')
('reference_to_opponent', 'number')
('reference_to_opponent', 'modal_verb')
('swear_words', 'personal_pronouns')
('swear_words', 'questions')
('swear_words', 'websites')
('swear_words', 'exclamation')
('swear_words', 'number')
('swear_words', 'modal_verb')
('personal_pronouns', 'questions')
('personal_pronouns', 'websites')
('personal_pronouns', 'exclamation')
('personal_pronouns', 'number')
('personal_pronouns', 'modal_verb')
('questions', 'websites')
('questions', 'exclamation')
('questions', 'number')
('questions', 'modal_verb')
('websites', 'exclamation')
('websites', 'n

In [7]:
linguistic_transformer_dict = {
        'length': transformer_get_length,
        'reference_to_opponent': transformer_get_reference_to_opponent,
        'swear_words': transformer_get_swear_words,
        'personal_pronouns': transformer_get_personal_pronouns, 
        "questions":  transformer_get_questions,
        "websites":  transformer_get_websites,
        "exclamation":  transformer_get_exclamation,
        "number":  transformer_get_number,
        "modal_verb":   transformer_get_modal_verb
}

lexicon_transformer_dict = {
        'connotation': transformer_get_connotation_feature,
        'vad': transformer_get_vad_feature
}

ngram_transformer_dict = {
        'trigram': transformer_get_trigrams
}

users_transformer_dict = {
          'transformer_get_cosine_similarity '         : transformer_get_cosine_similarity ,
          'transformer_get_religious_align'         : transformer_get_religious_align,
          'transformer_get_political_align'         : transformer_get_political_align,
          'transformer_get_education_user_align'         : transformer_get_education_user_align,
          'transformer_get_party_user_align'         : transformer_get_party_user_align,
          'transformer_get_gender_user_align'         : transformer_get_gender_user_align,
          'transformer_get_ethnicity_user_align'         : transformer_get_ethnicity_user_align,
          'transformer_get_relationship_user_align'         : transformer_get_relationship_user_align

}

# Using only ngram 
# Using only ngram + lexicon 
# Using only ngram + linguistic 
# Using only ngram + users

# Using only ngram + lexicon + linguistic
# Using only ngram + lexicon + linguistic + users 

def get_trans_from_name(trans_names, trans_dict): 
    return [(name, trans_dict[name]) for name in trans_names]

def select_linguistic_transformer():
    for names in itertools.combinations(linguistic_transformer_dict, 2):
        name1, name2 = names
        yield [(name1, linguistic_transformer_dict[name1]),
              (name2, linguistic_transformer_dict[name2])]

def select_lexicon_transformer():
    for names in itertools.combinations(lexicon_transformer_dict, 1):
        name1 = names[0]
        yield [(name1, lexicon_transformer_dict[name1])]

def select_ngram_transformer():
    for names in itertools.combinations(ngram_transformer_dict, 1):
        name1 = names[0]
        yield [(name1, ngram_transformer_dict[name1])]

#         chagne iteration to 2 later on 
def select_users_transformer():
    for names in itertools.combinations(users_transformer_dict, 2):
        name1, name2 = names
        yield [(name1, users_transformer_dict[name1]),
              (name2, users_transformer_dict[name2])]

def names(trans): 
    return [item[0] for item in trans]

def ngram_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in [[]]: 
            for linguistic_trans in [[]]:
                for users_trans in [[]]:
                    train_report, val_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + [train_report, val_report]
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report", "val_report"])
    report_df.to_csv("./resources/report/ngram_only.csv")
    return report_df

def ngram_linguistic_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in [[]]: 
            for linguistic_trans in select_linguistic_transformer():
                for users_trans in [[]]:
                    train_report, val_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + [train_report, val_report]
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report", "val_report"])
    report_df.to_csv("./resources/report/ngram_linguistic_only.csv")
    return report_df

def ngram_lexicon_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in select_lexicon_transformer(): 
            for linguistic_trans in [[]]:
                for users_trans in [[]]:
                    train_report, val_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + [train_report, val_report]
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report", "val_report"])
    report_df.to_csv("./resources/report/ngram_lexicon_only.csv")
    return report_df

def ngram_lexicon_linguistic_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in select_lexicon_transformer(): 
            for linguistic_trans in select_linguistic_transformer():
                for users_trans in [[]]:
                    train_report, val_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + [train_report, val_report]
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report", "val_report"])
    report_df.to_csv("./resources/report/ngram_lexicon_linguistic_only.csv")
    return report_df

def ngram_lexicon_linguistic_users_only():
    selected_linguistic_trans = get_trans_from_name(['reference_to_opponent', 'swear_words'], 
                                                    linguistic_transformer_dict)
    selected_lexicon_trans = get_trans_from_name(['vad'], lexicon_transformer_dict)
    selected_ngram_trans = get_trans_from_name(['trigram'], ngram_transformer_dict)
    
    report_df = []
    for ngram_trans in [selected_ngram_trans]: 
        for lexicon_trans in [selected_lexicon_trans]: 
            for linguistic_trans in [selected_linguistic_trans]:
                for users_trans in select_users_transformer():
                    print(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    train_report, val_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + [train_report, val_report]
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report", "val_report"])
    report_df.to_csv("./resources/report/ngram_lexicon_linguistic_users_only.csv")
    return report_df

def run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans):
    linguistic_lexicon_trans = FeatureUnion(
        linguistic_trans + lexicon_trans + ngram_trans
    )

    side_trans = Pipeline(
        [
            ('ngram', transformer_get_unigrams), 
            ('linguistic_lexicon', linguistic_lexicon_trans)
        ]
    )

    pro_trans = Pipeline(
        [
            ('get_pro', transformer_get_pro),
            ('side_trans', side_trans) 
        ]
    )

    con_trans = Pipeline(
        [
            ('get_con', transformer_get_con),
            ('side_trans', side_trans) 
        ]
    )

    both_trans = FeatureUnion(
        [
            ('pro_feature', pro_trans),
            ('con_feature', con_trans)
        ] + users_trans
    #         ('cosine_similarity', transformer_get_cosine_similarity)
    )

    big_trans = Pipeline(
        [
            ('separate_document', transformer_separate_document), 
            ('get_both_features', both_trans),
            ('logistic_regression', sklearn.linear_model.LogisticRegression())
        ]
    )

    clf = big_trans.fit(df_train, get_winner(df_train))

    train_report = classification_report(get_winner(df_train), 
                                       clf.predict(df_train), output_dict=True)
    print("train accuracy", train_report['accuracy'])
    val_report = classification_report(get_winner(df_val), 
                                       clf.predict(df_val), output_dict=True)
    print("val accuracy", val_report['accuracy'])
    return train_report, val_report 


In [8]:
ngram_lexicon_linguistic_users_only()

[('reference_to_opponent', Transformer_get_reference_to_opponent()), ('swear_words', Transformer_get_swear_words())] [('vad', Transformer_get_vad_feature())] [('trigram', Transformer_get_trigrams())] [('transformer_get_cosine_similarity ', Transformer_get_cosine_similarity()), ('transformer_get_religious_align', Transformer_get_religious_align())]


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.8536432160804021
val accuracy 0.7443609022556391
[('reference_to_opponent', Transformer_get_reference_to_opponent()), ('swear_words', Transformer_get_swear_words())] [('vad', Transformer_get_vad_feature())] [('trigram', Transformer_get_trigrams())] [('transformer_get_cosine_similarity ', Transformer_get_cosine_similarity()), ('transformer_get_political_align', Transformer_get_political_align())]


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.8435929648241206
val accuracy 0.7568922305764411
[('reference_to_opponent', Transformer_get_reference_to_opponent()), ('swear_words', Transformer_get_swear_words())] [('vad', Transformer_get_vad_feature())] [('trigram', Transformer_get_trigrams())] [('transformer_get_cosine_similarity ', Transformer_get_cosine_similarity()), ('transformer_get_education_user_align', Transformer_get_any_user_align(column='education'))]


...

/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.8373115577889447
val accuracy 0.7443609022556391
[('reference_to_opponent', Transformer_get_reference_to_opponent()), ('swear_words', Transformer_get_swear_words())] [('vad', Transformer_get_vad_feature())] [('trigram', Transformer_get_trigrams())] [('transformer_get_ethnicity_user_align', Transformer_get_any_user_align(column='ethnicity')), ('transformer_get_relationship_user_align', Transformer_get_any_user_align(column='relationship'))]


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.8448492462311558
val accuracy 0.7418546365914787


,ngram_trans,lexicon_trans,linguistic_trans,users_trans,train_report,val_report
0,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.8474059003051883, 'recal...","{'0': {'precision': 0.7280334728033473, 'recal..."
1,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.8492146596858638, 'recal...","{'0': {'precision': 0.7394957983193278, 'recal..."
2,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.8348348348348348, 'recal...","{'0': {'precision': 0.714859437751004, 'recall..."
3,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.857727737973388, 'recall...","{'0': {'precision': 0.7224489795918367, 'recal..."
4,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.8614270941054809, 'recal...","{'0': {'precision': 0.7165991902834008, 'recal..."
5,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.8443298969072165, 'recal...","{'0': {'precision': 0.7387755102040816, 'recal..."
6,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_cosine_similarity , transform...","{'0': {'precision': 0.8520408163265306, 'recal...","{'0': {'precision': 0.7377049180327869, 'recal..."
7,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_religious_align, transformer_...","{'0': {'precision': 0.8383110195674562, 'recal...","{'0': {'precision': 0.75, 'recall': 0.83886255..."
8,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_religious_align, transformer_...","{'0': {'precision': 0.8394160583941606, 'recal...","{'0': {'precision': 0.7565217391304347, 'recal..."
9,[trigram],[vad],"[reference_to_opponent, swear_words]","[transformer_get_religious_align, transformer_...","{'0': {'precision': 0.8467078189300411, 'recal...","{'0': {'precision': 0.7283950617283951, 'recal..."


In [90]:
linguistic_trans = ['reference_to_opponent', 'swear_words']
lexicon_trans    = ['vad']
ngram_trans = ['trigram']
users_trans = []


ngram_trans = get_trans_from_name(ngram_trans, ngram_transformer_dict)
users_trans = get_trans_from_name(users_trans, users_transformer_dict)
linguistic_trans = get_trans_from_name(linguistic_trans, linguistic_transformer_dict)
lexicon_trans    = get_trans_from_name(lexicon_trans   , lexicon_transformer_dict)

clf = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
ling_pred_val = clf.predict(df_val)

/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [91]:
linguistic_trans = []
lexicon_trans    = []
ngram_trans = ['trigram']
users_trans = []

def get_trans_from_name(trans_names, trans_dict): 
    return [(name, trans_dict[name]) for name in trans_names]

ngram_trans = get_trans_from_name(ngram_trans, ngram_transformer_dict)
users_trans = get_trans_from_name(users_trans, users_transformer_dict)
linguistic_trans = get_trans_from_name(linguistic_trans, linguistic_transformer_dict)
lexicon_trans    = get_trans_from_name(lexicon_trans   , lexicon_transformer_dict)

clf = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
ngram_pred_val = clf.predict(df_val)

In [164]:
class Transformer_get_vad_feature(BaseEstimator, TransformerMixin):
    def __init__(self):
        NRC_LEXICON_VAD = "./resources/lexica/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt"
        
        self.df_vad = pd.read_csv(NRC_LEXICON_VAD, sep="	", header=None)
        self.df_vad.columns = ["word", "valence", "arousal", "dominance"]
        self.df_vad = self.df_vad.dropna()
        self.df_vad = self.df_vad.set_index("word")
        self.df_vad["valence"] = self.df_vad["valence"].astype('category')
        self.df_vad["arousal"] = self.df_vad["arousal"].astype('category')
        self.df_vad["dominance"] = self.df_vad["dominance"].astype('category')
        
    def fit(self, X, y=None):
        unigram_vectorizer = X['unigram_vectorizer']
        
        word_vad = self.df_vad.index
        word_vector_vad = unigram_vectorizer.transform(word_vad)
        self.matrix_vad = word_vector_vad.T @ self.df_vad 
        return self 
    
    def transform(self, X, y=None):
        gram_pro = X['unigram']
        feature_pro = gram_pro @ self.matrix_vad / np.sum(gram_pro)
        return feature_pro

document_side = get_text_by_side(df_train)

In [171]:
transformer_separate_document = Transformer_separate_document()
X = transformer_separate_document.fit_transform(df_train)
X = transformer_get_pro.fit_transform(X)
X = transformer_get_unigrams.fit_transform(X)
vad = transformer_get_vad_feature.fit_transform(X)

In [201]:
transformer_separate_document = Transformer_separate_document()
X = transformer_separate_document.fit_transform(df_train)
X = transformer_get_con.fit_transform(X)
X = transformer_get_unigrams.fit_transform(X)
vad = transformer_get_vad_feature.fit_transform(X)

In [236]:
transformer_get_vad_feature.df_vad.sort_values(by=['valence'], ascending=False).loc['rape', :]

valence      0.062
arousal      0.856
dominance    0.527
Name: rape, dtype: float64

In [233]:
con_df = pd.concat([X['df'], pd.DataFrame(data=vad, columns=['valence', 'arousal', 'dominance'])], axis=1)
con_df = con_df.sort_values(by=['dominance'], ascending=False)
con_df.loc[7]

KeyError: 'max'

In [223]:
print(df_train.loc[7, 'title'])

Evidence for God


In [54]:
ngram_only()
ngram_lexicon_only()
ngram_linguistic_only()
ngram_lexicon_linguistic_only()
# ngram_lexicon_linguistic_users_only()

train accuracy 0.8881909547738693
val accuracy 0.7268170426065163
train accuracy 0.7267587939698492
val accuracy 0.7293233082706767


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.8077889447236181
val accuracy 0.7343358395989975


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.8304020100502513
val accuracy 0.7418546365914787
train accuracy 0.8115577889447236
val accuracy 0.7393483709273183


...

/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.7418341708542714
val accuracy 0.7142857142857143


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train accuracy 0.7405778894472361
val accuracy 0.7293233082706767


,ngram_trans,lexicon_trans,linguistic_trans,users_trans,train_report,val_report
0,[trigram],[connotation],"[length, reference_to_opponent]",[],"{'0': {'precision': 0.7634989200863931, 'recal...","{'0': {'precision': 0.712, 'recall': 0.8436018..."
1,[trigram],[connotation],"[length, swear_words]",[],"{'0': {'precision': 0.7643865363735071, 'recal...","{'0': {'precision': 0.692, 'recall': 0.8199052..."
2,[trigram],[connotation],"[length, personal_pronouns]",[],"{'0': {'precision': 0.7667386609071274, 'recal...","{'0': {'precision': 0.7235772357723578, 'recal..."
3,[trigram],[connotation],"[length, questions]",[],"{'0': {'precision': 0.7663043478260869, 'recal...","{'0': {'precision': 0.7016129032258065, 'recal..."
4,[trigram],[connotation],"[length, websites]",[],"{'0': {'precision': 0.7603485838779956, 'recal...","{'0': {'precision': 0.7051792828685259, 'recal..."
...,...,...,...,...,...,...
67,[trigram],[vad],"[websites, number]",[],"{'0': {'precision': 0.7698833510074231, 'recal...","{'0': {'precision': 0.7188755020080321, 'recal..."
68,[trigram],[vad],"[websites, modal_verb]",[],"{'0': {'precision': 0.774331550802139, 'recall...","{'0': {'precision': 0.717741935483871, 'recall..."
69,[trigram],[vad],"[exclamation, number]",[],"{'0': {'precision': 0.7441176470588236, 'recal...","{'0': {'precision': 0.6752767527675276, 'recal..."
70,[trigram],[vad],"[exclamation, modal_verb]",[],"{'0': {'precision': 0.7584442169907881, 'recal...","{'0': {'precision': 0.685823754789272, 'recall..."


In [9]:
df_train_partition_1 = df_train[df_train.category == "Religion"]
df_train_partition_0 = df_train[df_train.category != "Religion"]
df_val_partition_1 = df_val[df_val.category == "Religion"]
df_val_partition_0 = df_val[df_val.category != "Religious"]

In [10]:
REPORT_DIRECTORY = "./resources/report/partition/"

# linguistic_transformer_dict = {
#         'length': transformer_get_length,
#         'reference_to_opponent': transformer_get_reference_to_opponent,
#         'swear_words': transformer_get_swear_words,
#         'personal_pronouns': transformer_get_personal_pronouns, 
#         "questions":  transformer_get_questions,
#         "websites":  transformer_get_websites,
#         "exclamation":  transformer_get_exclamation,
#         "number":  transformer_get_number,
#         "modal_verb":   transformer_get_modal_verb
# }

# lexicon_transformer_dict = {
#         'connotation': transformer_get_connotation_feature,
#         'vad': transformer_get_vad_feature
# }

# ngram_transformer_dict = {
#         'trigram': transformer_get_trigrams
# }

# users_transformer_dict = {
#         'cosine_similarity': transformer_get_cosine_similarity
# }

# Using only ngram 
# Using only ngram + lexicon 
# Using only ngram + linguistic 
# Using only ngram + users

# Using only ngram + lexicon + linguistic
# Using only ngram + lexicon + linguistic + users 

def select_linguistic_transformer():
    for names in itertools.combinations(linguistic_transformer_dict, 2):
        name1, name2 = names
        yield [(name1, linguistic_transformer_dict[name1]),
              (name2, linguistic_transformer_dict[name2])]

def select_lexicon_transformer():
    for names in itertools.combinations(lexicon_transformer_dict, 1):
        name1 = names[0]
        yield [(name1, lexicon_transformer_dict[name1])]

def select_ngram_transformer():
    for names in itertools.combinations(ngram_transformer_dict, 1):
        name1 = names[0]
        yield [(name1, ngram_transformer_dict[name1])]

#         chagne iteration to 2 later on 
def select_users_transformer():
    for names in itertools.combinations(users_transformer_dict, 2):
        name1, name2 = names
        yield [(name1, users_transformer_dict[name1]),
              (name2, users_transformer_dict[name2])]

def names(trans): 
    return [item[0] for item in trans]

def ngram_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in [[]]: 
            for linguistic_trans in [[]]:
                for users_trans in [[]]:
                    partition_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + partition_report
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report_partition_0", "val_report_partition_0", 
                                                "train_report_partition_1", "val_report_partition_1"])
    report_file_name = "ngram_only.csv"
    report_df.to_csv(REPORT_DIRECTORY + report_file_name)
    return report_df

def ngram_linguistic_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in [[]]: 
            for linguistic_trans in select_linguistic_transformer():
                for users_trans in [[]]:
                    partition_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + partition_report
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report_partition_0", "val_report_partition_0", 
                                                "train_report_partition_1", "val_report_partition_1"])
    
    report_file_name = "ngram_linguistic_only.csv"
    report_df.to_csv(REPORT_DIRECTORY + report_file_name)
    return report_df

def ngram_lexicon_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in select_lexicon_transformer(): 
            for linguistic_trans in [[]]:
                for users_trans in [[]]:
                    partition_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + partition_report
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report_partition_0", "val_report_partition_0", 
                                                "train_report_partition_1", "val_report_partition_1"])
    
    report_file_name = "ngram_lexicon_only.csv"
    report_df.to_csv(REPORT_DIRECTORY + report_file_name)
    return report_df

def ngram_lexicon_linguistic_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in select_lexicon_transformer(): 
            for linguistic_trans in select_linguistic_transformer():
                for users_trans in [[]]:
                    partition_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + partition_report
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report_partition_0", "val_report_partition_0", 
                                                "train_report_partition_1", "val_report_partition_1"])
    
    report_df.to_csv("./resources/report/partitionngram_lexicon_linguistic_only.csv")
    return report_df

def ngram_lexicon_linguistic_users_only():
    report_df = []
    for ngram_trans in select_ngram_transformer(): 
        for lexicon_trans in select_lexicon_transformer(): 
            for linguistic_trans in select_linguistic_transformer():
                for users_trans in select_users_transformer():
                    
                    partition_report = run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans)
                    row = list(map(names, (ngram_trans, lexicon_trans, linguistic_trans, users_trans))) \
                            + partition_report
                    report_df.append(row)


    report_df = pd.DataFrame(report_df, columns=["ngram_trans", "lexicon_trans", "linguistic_trans", "users_trans",
                                                "train_report_partition_0", "val_report_partition_0", 
                                                "train_report_partition_1", "val_report_partition_1"])
    
    report_file_name = "/all_selection.csv"
    report_df.to_csv(REPORT_DIRECTORY + report_file_name)
    return report_df

def run_pipeline(linguistic_trans, lexicon_trans, ngram_trans, users_trans):
    linguistic_lexicon_trans = FeatureUnion(
        linguistic_trans + lexicon_trans + ngram_trans
    )

    side_trans = Pipeline(
        [
            ('ngram', transformer_get_unigrams), 
            ('linguistic_lexicon', linguistic_lexicon_trans)
        ]
    )

    pro_trans = Pipeline(
        [
            ('get_pro', transformer_get_pro),
            ('side_trans', side_trans) 
        ]
    )

    con_trans = Pipeline(
        [
            ('get_con', transformer_get_con),
            ('side_trans', side_trans) 
        ]
    )

    both_trans = FeatureUnion(
        [
            ('pro_feature', pro_trans),
            ('con_feature', con_trans)
        ] + users_trans
    #         ('cosine_similarity', transformer_get_cosine_similarity)
    )

    big_trans = Pipeline(
        [
            ('separate_document', transformer_separate_document), 
            ('get_both_features', both_trans),
            ('logistic_regression', sklearn.linear_model.LogisticRegression())
        ]
    )

    clf_partition_0 = big_trans.fit(df_train_partition_0, get_winner(df_train_partition_0))

    train_report_partition_0 = classification_report(get_winner(df_train_partition_0), 
                                       clf_partition_0.predict(df_train_partition_0), output_dict=True)
#     print("train accuracy", train_report['accuracy'])
    val_report_partition_0 = classification_report(get_winner(df_val_partition_0), 
                                       clf_partition_0.predict(df_val_partition_0), output_dict=True)
#     print("val accuracy", val_report['accuracy'])
    clf_partition_1 = big_trans.fit(df_train_partition_1, get_winner(df_train_partition_1))

    train_report_partition_1 = classification_report(get_winner(df_train_partition_1), 
                                       clf_partition_1.predict(df_train_partition_1), output_dict=True)
#     print("train accuracy", train_report['accuracy'])
    val_report_partition_1 = classification_report(get_winner(df_val_partition_1), 
                                       clf_partition_1.predict(df_val_partition_1), output_dict=True)
    print("val religious accuracy", val_report_partition_1['accuracy'])
    print("val non religious accuracy", val_report_partition_0['accuracy'])
    
    return [train_report_partition_0, val_report_partition_0, train_report_partition_1, val_report_partition_1]

In [ ]:
ngram_lexicon_linguistic_users_only()

/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org

val religious accuracy 0.6451612903225806
val non religious accuracy 0.7343358395989975


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org

val religious accuracy 0.6129032258064516
val non religious accuracy 0.7543859649122807


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org

...

val religious accuracy 0.6344086021505376
val non religious accuracy 0.7368421052631579


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org

val religious accuracy 0.6666666666666666
val non religious accuracy 0.7493734335839599


/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org

val religious accuracy 0.6666666666666666
val non religious accuracy 0.7518796992481203


In [85]:
df_val.shape


(399, 9)

In [ ]:
from autocorrect import Speller
spell = Speller()
spell.existing("I'm not sleapy and tehre is no place I'm giong to.")

In [ ]:
get_modal_verb(document_side, unigram_vectorizer)

In [4]:
def get_jsonl(path):

    with open(path) as json_file:
        json_list = list(json_file)

    data_list = []
    for json_str in json_list:
        data_list.append(json.loads(json_str))

    return pd.DataFrame(data_list)
def get_texts(df):
    '''
    Return a list of statements in df without differentiating the side of the speaker
    '''

    texts = []
    for round in df.loc[:, 'rounds']:
        for sub_round in round:
            for speech in sub_round:
                texts.append(speech['text'])

    return texts

def get_text_by_side(df): 
    '''
    Return a list of documents where each document contains all text on one side in a 
    single debate
    
    text = [[Pro statement 1, Pro statement 2, ... Pro statement n],
            [Con statement 1, Con statement 2, ... Con statement m]]
            where n, m is the total number of statements from Pro and Con side across
            all debates

    size: [n x 2 x # statements in each debate]
    '''

    text = []
    for round in df.loc[:, 'rounds']:
        round_text = collections.defaultdict(list)

        for sub_round in round:
            for speech in sub_round: 
                round_text[speech['side']].append(speech['text'])

        
        text.append(["".join(round_text['Pro']), "".join(round_text['Con'])])

    return np.array(text)

def get_text_by_side_by_speech(df): 
    '''
    Return a list of documents where each document contains all text on one side in a 
    single debate
    
    text = [[Pro statement 1, Pro statement 2, ... Pro statement n],
            [Con statement 1, Con statement 2, ... Con statement m]]
            where n, m is the total number of statements from Pro and Con side across
            all debates

    size: [n x 2 x # statements in each debate]
    '''

    text = []
    for round in df.loc[:, 'rounds']:
        round_text = collections.defaultdict(list)

        for sub_round in round:
            for speech in sub_round: 
                round_text[speech['side']].append(speech['text'])

        
        text.append(["".join(round_text['Pro']), "".join(round_text['Con'])])
        

    return np.array(text)


def get_ngram_feature(document_side, vectorizer): 
    '''
    Return the ngram features associated with a single debate

    For pro side, each document is defined as a string that contains all the statements 
    from the pro side in a single debate (across different subrounds). Con side is 
    similarly defined. 

    return [[Pro side n gram vector, Con side n gram vector for 1 debate],
            [Pro side n gram vector, Con side n gram vector for 2 debate],
            ...]

            size: [n, 2 x ngram count]
    
    Pro side and con side n gram vector are concatenated.
    '''

    pro_document = document_side[:, 0]
    con_document = document_side[:, 1]

    pro_feature = vectorizer.transform(pro_document)
    con_feature = vectorizer.transform(con_document)
    return sparse.hstack([pro_feature, con_feature])   

def get_debate_feature(df):
    '''
    Return the debate feature such as category, pro_debator user name, etc

    feature: [n, # of features] 
    '''
    feature_name = ['category']
    feature = []

    for name in feature_name: 
        # TODO: check for data type of the column. If non-numeric, then do this
        # otherwise, use the numerical data
        encoding, unique_feature_val = pd.factorize(df[name])
        feature.append(encoding)

    return np.reshape(np.array(feature), [-1, len(feature_name)])

def get_winner(df): 
    '''
    Cons gets mapped to 0 and pro gets mapped to 1
    '''
    return np.array(df.loc[:, "winner"].replace({"Con": 0, "Pro": 1}))


# Model 2 - lex feature, debate feature, n-gram feature
This model should use
1. word ngrams
2. lexicon based features: implement lexicon based features for a lexicon of your choice
   1. Connotation lexicon
   2. NRC-VAD lexicon
   3. How you extract features is part of the desgin decision that you need to make. One simple example for lexical features could be counting how many words in each debaters language appear in the corresponding lexicon. 

TODO: 
1. Read connotation - 1 file
2. NRC features - 2 files 

In [5]:
# 1. Read connotation - 1 file
# 2. NRC features - 2 files 
CONNOTATION = "./resources/lexica/connotation_lexicon_a.0.1.csv"
NRC_LEXICON_VAD = "./resources/lexica/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt"
NRC_LEXICON_SORTED_VALENCE = "./resources/lexica/NRC-VAD-Lexicon-Aug2018Release/OneFilePerDimension/v-scores.txt"
NRC_LEXICON_SORTED_AROUSAL = "./resources/lexica/NRC-VAD-Lexicon-Aug2018Release/OneFilePerDimension/a-scores.txt"
NRC_LEXICON_SORTED_DOMINANCE = "./resources/lexica/NRC-VAD-Lexicon-Aug2018Release/OneFilePerDimension/d-scores.txt"

df_connotation = pd.read_csv(CONNOTATION, sep=",|_", header=None)
df_connotation.columns = ["word", "pos", "connotation"] # word, part of speech, connotation
df_connotation = df_connotation.dropna() # There are five words in the connotation that are nan 
df_connotation = df_connotation.set_index("word")
df_connotation["pos"] = df_connotation["pos"].astype('category')
df_connotation = df_connotation.drop(columns=["pos"]) # drop the part of speech classification because we can't use it now 
df_connotation["connotation"] = df_connotation["connotation"].astype('category')
df_connotation = pd.get_dummies(df_connotation)

df_nrc_vad = pd.read_csv(NRC_LEXICON_VAD, sep="	", header=None)
df_nrc_vad.columns = ["word", "valence", "arousal", "dominance"]
df_nrc_vad = df_nrc_vad.dropna()
df_nrc_vad = df_nrc_vad.set_index("word")
df_nrc_vad["valence"] = df_nrc_vad["valence"].astype('category')
df_nrc_vad["arousal"] = df_nrc_vad["arousal"].astype('category')
df_nrc_vad["dominance"] = df_nrc_vad["dominance"].astype('category')

/Users/xhe/opt/anaconda3/envs/env_nlp/lib/python3.6/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [6]:
# Get features and labels for traininig and validation 
unigram_vectorizer = CountVectorizer()

# Generate the corpus for vectotrizer to fit on 
document_train_side = get_text_by_side(df_train)
document_val_side = get_text_by_side(df_val)
document_train = [side[0] + side[1] for side in document_train_side]
document_val = [side[0] + side[1] for side in document_val_side]

# The vectorizer trains all all the textual corpus regardless of the side 
# of the debate 
unigram_vectorizer.fit(document_train)

# Get the feature vector of a sentence using ngram @ matrix_connotation
# Creating the matrix 
word_connotation = df_connotation.index
word_vector_connotation = unigram_vectorizer.transform(word_connotation)
matrix_connotation = word_vector_connotation.T @ df_connotation
matrix_connotation_no_neutral = word_vector_connotation.T @ df_connotation.drop(columns=["connotation_neutral"])

word_vad = df_nrc_vad.index
word_vector_vad = unigram_vectorizer.transform(word_vad)
matrix_vad = word_vector_vad.T @ df_nrc_vad
# For words with mulitple part of speech, we are counting the total
# sum across all part of speech of that word for each feature 

# Get label 
label_train = get_winner(df_train)
label_val = get_winner(df_val)

y_train = np.array(label_train)
y_val = np.array(label_val)

# Get more grams 
trigram_vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=0.2, stop_words='english', ngram_range=(1,3))
trigram_vectorizer.fit(document_train)

TfidfVectorizer(max_df=0.8, min_df=0.2, ngram_range=(1, 3),
                stop_words='english', sublinear_tf=True)

In [ ]:
# ================== you can run experiments here ======================
# Get all TRAINING features:
# Get the documents on pro and con side so that we can forming feature 
# vector on both sides for training 

trigram_train = get_ngram_feature(document_side=document_train_side, vectorizer=trigram_vectorizer)
# ============= using raw number counts of the feature ==========================
# feature_connotation_train = get_connotation_feature(document_side=document_train_side,
#                                                         matrix_connotation=matrix_connotation,
#                                                         vectorizer=unigram_vectorizer)
# feature_vad_train = get_vad_feature(document_side=document_train_side,
#                                                         matrix_vad=matrix_vad,
#                                                         vectorizer=unigram_vectorizer)
# feature_train = sparse.hstack([trigram_train, feature_connotation_train, 
#                     feature_vad_train])

# ============= using percentage counts of the feature ==========================
# feature_connotation_pct_train = get_connotation_percentage_feature(document_train_side, matrix_connotation, unigram_vectorizer)
# feature_vad_pct_train = get_vad_percentage_feature(document_train_side, matrix_vad, unigram_vectorizer)
# feature_train = sparse.hstack([trigram_train, feature_connotation_pct_train, 
#                     feature_vad_pct_train])

# ============= using log counts of the feature ==========================
# feature_connotation_ln_train = get_connotation_ln_feature(document_train_side, matrix_connotation, unigram_vectorizer)
# feature_vad_ln_train = get_vad_ln_feature(document_train_side, matrix_vad, unigram_vectorizer)
# feature_train = sparse.hstack([trigram_train, feature_connotation_ln_train, 
#                     feature_vad_ln_train])

# ============= using percentage counts of the feature without neutral connotation ==========================
feature_connotation_pct_train = get_connotation_percentage_feature(document_train_side, matrix_connotation_no_neutral, unigram_vectorizer)
feature_vad_pct_train = get_vad_percentage_feature(document_train_side, matrix_vad, unigram_vectorizer)
feature_train = sparse.hstack([trigram_train, feature_connotation_pct_train, 
                    feature_vad_pct_train])

# Get all VALIDATION features:
trigram_val = get_ngram_feature(document_side=document_val_side, vectorizer=trigram_vectorizer)
# ============= using raw counts of of the feature ==========================
# feature_connotation_val = get_connotation_feature(document_side=document_val_side,
#                                                         matrix_connotation=matrix_connotation,
#                                                         vectorizer=unigram_vectorizer)
# feature_vad_val = get_vad_feature(document_side=document_val_side,
#                                                         matrix_vad=matrix_vad,
#                                                         vectorizer=unigram_vectorizer)
# feature_val = sparse.hstack([trigram_val, feature_connotation_val, 
#                     feature_vad_val])

# ============= using percentage count of of the feature ==========================
# feature_connotation_pct_val = get_connotation_percentage_feature(document_val_side, matrix_connotation, unigram_vectorizer)
# feature_vad_pct_val = get_vad_percentage_feature(document_val_side, matrix_vad, unigram_vectorizer)
# feature_train = sparse.hstack([trigram_train, feature_connotation_pct_train, 
#                     feature_vad_pct_train])
# feature_val = sparse.hstack([trigram_val, feature_connotation_pct_val, 
#                     feature_vad_pct_val])

# ============= using log counts of the feature ==========================
# feature_connotation_ln_val = get_connotation_ln_feature(document_val_side, matrix_connotation, unigram_vectorizer)
# feature_vad_ln_val = get_vad_ln_feature(document_val_side, matrix_vad, unigram_vectorizer)

# feature_val = sparse.hstack([trigram_val, feature_connotation_ln_val, 
#                     feature_vad_ln_val])

# ============= using percentage counts of the feature without neutral connotation ==========================
feature_connotation_pct_val = get_connotation_percentage_feature(document_val_side, matrix_connotation_no_neutral, unigram_vectorizer)
feature_vad_pct_val = get_vad_percentage_feature(document_val_side, matrix_vad, unigram_vectorizer)

feature_val = sparse.hstack([trigram_val, feature_connotation_pct_val, 
                    feature_vad_pct_val])

# Create model
clf = sklearn.linear_model.LogisticRegression()
clf.fit(feature_train, y_train)

In [ ]:
print(classification_report(y_train, clf.predict(feature_train)))
print(classification_report(y_val, clf.predict(feature_val)))

# Model 1 - Here is the model that only uses debate features and ngram features

In [ ]:
# Extracting texts from training and testing data
label_train = get_winner(df_train)
label_val = get_winner(df_val)

# Generate the corpus 
document_train = get_text_by_side(df_train)
document_val = get_text_by_side(df_val)

# Vectorization
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.9, min_df=0.1, stop_words='english', ngram_range=(1,3))
vectorizer.fit(document_train)

# Getting two sets of features - ngram and debate related features
ngram_feature_train = get_ngram_feature(df_train, vectorizer)
ngram_feature_val = get_ngram_feature(df_val, vectorizer)

debate_feature_train = get_debate_feature(df_train)
debate_feture_val = get_debate_feature(df_val)

# Combining two sets of features
X_train = sparse.hstack([debate_feature_train, ngram_feature_train])
X_val = sparse.hstack([debate_feture_val, ngram_feature_val])

y_train = np.array(label_train)
y_val = np.array(label_val)

In [ ]:
print('Sanity check')
print(df_train.shape[0], 'number of observations in the training set')
print(X_train.shape, 'number of observation x the size of ngram vectors in the training set')
print(y_train.shape, 'number of labels in the training set')
print(df_val.shape[0], 'number of observations in the validation set')
print(X_val.shape, 'number of observation x the size of ngram vectors in the validation set')
print(y_val.shape, 'number of labels in the validation set')


In [ ]:
# Building and training the model
clf = sklearn.linear_model.LogisticRegression()
clf.fit(ngram_feature_train, y_train)

print("Logistic Regression training set report:")
print(classification_report(y_train, clf.predict(ngram_feature_train), target_names=['Pro', 'Con']))
print(classification_report(y_val, clf.predict(ngram_feature_val), target_names=['Pro', 'Con']))

# %%

In [ ]:
X_val

In [ ]:
# Evaluating the model on the validation set
y_predicted = clf.predict(X_val_religion)
print("Logistic Regression testing set report:")
print(classification_report(y_val_religion, y_predicted, target_names=['Pro', 'Con']))

print("Accuracy score: ",accuracy_score(y_val_religion, y_predicted))
print("Balanced accuracy score: ",accuracy_score(y_val_religion, y_predicted))

plot_confusion_matrix(clf, X_val, y_predicted)

In [ ]:
# Tuning ngram models over max_df and min_df
def search_max_df_min_df(df_train, df_val):
    highest_acc, best_min_df, best_max_df = 0, -1, -1
    report = {}
    for min_df in np.arange(0, 1, 0.1):
        for diff in np.arange(0.1, 1 - min_df, 0.1):
            max_df = min_df + diff

            vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=max_df, min_df=min_df, stop_words='english', ngram_range=(1,3))
            document_train = get_text_by_side(df_train)
            vectorizer.fit(document_train)
            X_train, y_train = get_all_feature_label(df_train, vectorizer)
            X_val, y_val = get_all_feature_label(df_val, vectorizer)

            clf = sklearn.linear_model.LogisticRegression()
            clf.fit(X_train, y_train)
            
            print("====================================")

            y_predicted = clf.predict(X_val)
            print("Logistic Regression testing set report:")
            report[(min_df, max_df)] = classification_report(y_val, y_predicted, target_names=['Pro', 'Con'], output_dict=True)
            acc = accuracy_score(y_val, y_predicted)

            print("max_df: {}, min_df: {}, accuracy: {}".format(max_df, min_df, acc))

            if acc > highest_acc:
                highest_acc, best_min_df, best_max_df = acc, min_df, max_df

    print("************ best min_df, best max_df, acc", best_min_df, best_max_df, highest_acc)
    return report



In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
max_acc, best_min_df, best_max_df = 0, -1, -1
gram3_report = report

for key, val in report.items():
    print("====================")
    print(key)
    print(val)

# The best min df and the best max df are (0.2, 0.8) with validation accuracy of 0.76





One way of achieving this is to create two n-gram models. One n-gram model outputs features
for religious topics and another n-gram model outputs features for non-religious topics.
By limiting the corpus within their topics, the Tf_idf scores may better reflect the 
proper weighting. For example, certain words that might only appear in winning relgious debates
but also appear in all other losing debates may now have a significantly different score from 
words that appear in only losing religous debates but appear in all other winning debates. 
Previously, these two sets of words would have similar tf_idf score but are not helpful 
towards predicting winning debates because their prediciton power within relgious topic is
diluted by the non-religous topics. By limiting the corpus scope, we can see that these 
words become helpful in both religous and non-relgious debates.

TODO:
1. Define a Tfidfvectorizer for both religous and non-religious topics
2. Train the vectorizer using their respective subsets
3. Depending the topic of the new data, we should use the two models conditionally

In [ ]:
# Partition the data sets
df_train_religion = df_train.loc[df_train.category == "Religion" ,:]
df_train_other = df_train.loc[df_train.category != "Religion" ,:]
df_val_religion = df_val.loc[df_val.category == "Religion" ,:]
df_val_other = df_val.loc[df_val.category != "Religion" ,:]

In [ ]:
print("Sanity check")
print(df_train_religious.shape)
print(df_train_other.shape)
print(df_train.shape)
print("validation set")
print(df_val_religious.shape)
print(df_val_other.shape)
print(df_val.shape)

In [ ]:
print(X_train_religion.shape)
print(X_val_religion.shape)

In [ ]:
search_max_df_min_df(df_train_religion, df_val_religion)
search_max_df_min_df(df_train_other, df_val_other)

In [ ]:
# Set up the vectorizer
vectorizer_religion = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=0, stop_words='english', ngram_range=(1,3))
document_train_religion = get_text_by_side(df_train_religion)
vectorizer_religion.fit(document_train_religion)
X_train_religion, y_train_religion = get_all_feature_label(df_train_religion, vectorizer_religion)
X_val_religion, y_val_religion = get_all_feature_label(df_val_religion, vectorizer_religion)
report_religion = search_max_df_min_df(X_train_religion, y_train_religion, X_val_religion, y_val_religion)

vectorizer_other = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=0, stop_words='english', ngram_range=(1,3))
document_train_other = get_text_by_side(df_train_other)
vectorizer_other.fit(document_train_other)
X_train_other, y_train_other = get_all_feature_label(df_train_other, vectorizer_other)
X_val_other, y_val_other = get_all_feature_label(df_val_other, vectorizer_other)
report_other = search_max_df_min_df(X_train_other, y_train_other, X_val_other, y_val_other)